In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
test_file = r'D:\ShiyuanDuan\SleepStudy\outputs\sub001.parquet'
df = pd.read_parquet(test_file).dropna()
epochs = df['Epoch'].unique()



In [3]:
data = df[['x','y','z','ma_hr', 'ma_rr','chest_orientation','Epoch', 'QuantizedStage']]

data = data.groupby('Epoch').apply(lambda x: x.values)

In [4]:
data

Epoch
<Epoch>-      [[-7296.0, 2683.0, 2384.0, 50.32356387626972, ...
<Epoch>1      [[381.0, -8263.0, -88.0, 66.24104237794107, 20...
<Epoch>10     [[10.0, -8109.0, 1666.0, 71.35595289304665, 23...
<Epoch>100    [[-3893.0, 5737.0, 4168.0, 57.27482228381032, ...
<Epoch>101    [[-3917.0, 6116.0, 3666.0, 58.981202869452815,...
                                    ...                        
<Epoch>95     [[-4041.0, 5310.0, 4602.0, 58.17330103493028, ...
<Epoch>96     [[-4020.0, 5710.0, 3835.0, 66.42293002834604, ...
<Epoch>97     [[-4124.0, 5786.0, 3709.0, 56.802009589406175,...
<Epoch>98     [[-4019.0, 5850.0, 3862.0, 60.81365754811682, ...
<Epoch>99     [[-4136.0, 6129.0, 3377.0, 59.38465722070431, ...
Length: 851, dtype: object

In [5]:
npval = np.stack([x for x in data if x.shape==(30000,8)])

In [11]:
npval[0]

array([[10.0, -8109.0, 1666.0, ..., -0.0702962670861917, '<Epoch>10', 0],
       [10.0, -8109.0, 1647.0, ..., -0.0703407845149495, '<Epoch>10', 0],
       [9.0, -8109.0, 1631.0, ..., -0.0703853019437072, '<Epoch>10', 0],
       ...,
       [134.0, -8148.0, 1406.0, ..., -0.1070332032574969, '<Epoch>10', 0],
       [135.0, -8148.0, 1430.0, ..., -0.1070625169208051, '<Epoch>10', 0],
       [135.0, -8148.0, 1406.0, ..., -0.1070918305841132, '<Epoch>10', 0]],
      dtype=object)

In [82]:
np.median(npval[:, :,-1], axis=1).astype(np.float32)

array([0., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2., 2.,
       2., 2., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 0.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 0., 3., 3., 0., 0., 1., 2., 2., 2.,
       2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 0., 0., 3., 3., 3., 3., 3., 3., 3., 0.,
       0., 0., 0., 0., 1., 1., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 3.,
       3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2.,
       0., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 0.,
       3., 3., 3., 2., 2.

In [2]:
import yaml

with open("../conf/config_train.yaml", "r") as file:
    config = yaml.safe_load(file)

train_file_paths = config["data"]["train"]
val_file_paths = config["data"]["val"]
test_file_paths = config["data"]["test"]

model_name = config["training"]["model"]
batch_size = config["training"]["batch_size"]
epochs = config["training"]["epochs"]
learning_rate = config["training"]["learning_rate"]
weight_decay = config["training"]["weight_decay"]
save_dir = config["training"]["save_dir"]


In [2]:
from data import prepare_dataloaders, custom_collate_fn


In [3]:
import yaml

with open("../conf/config_train.yaml", "r") as file:
    config = yaml.safe_load(file)
train, val, test = prepare_dataloaders(config=config)

Loading Test Data: 100%|██████████| 1/1 [00:26<00:00, 26.75s/it]


In [4]:
torch.unsqueeze(torch.rand(30000, 6), dim=0).size()

torch.Size([1, 30000, 6])

In [5]:
data_loader = DataLoader(train, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)

In [6]:
for d, y in train:
    print(d.size())
    print(y.size())

torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([16, 1, 30000, 6])
torch.Size([16])
torch.Size([1